# Enzyme-Target Prediction

This notebook is a demonstration for predicting potential interactions between enzymes and target molecules. This approach leverages the principles of quantum mechanics to solve optimization problems by formulating them as QUBO models

**Idea** is provided by: **Andreas Rossos** aka [@zmokuspokus in Discord]

**Dataset** is provided by: **MetXBioDB Metabolite Biotransformations**

**Code** is provided by: **Samer Rahmeh** aka [@samgr55 in Discord]

Enzyme-Target Prediction - QUBO/Ising ---- Repo is available here : https://github.com/samgr55/Enzyme-TargetPrediction_QUBO-Ising

# **Importing Libraries**

In [14]:
import dimod
from dimod import BinaryQuadraticModel, SimulatedAnnealingSampler
import pandas as pd
from collections import defaultdict, Counter
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem ,Descriptors, Draw
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
#import dynex

# **Loading Datasets**

In [15]:
df = pd.read_csv('Transformations.csv', encoding='latin1', nrows=1000) 
df1 = pd.read_csv('substances.csv', encoding='latin1', nrows=1000) 
df2 = pd.read_csv('metabolicdb.csv', encoding='latin1', nrows=1000) 

### Check the datatypes

In [16]:
print("Data types in df:")
print(df[['Predecessor_CID', 'Successor_CID', 'Enzyme']].dtypes)
print("\nData types in df1:")
print(df1[['InChI', 'PubChem_CID', 'SMILES']].dtypes)
print("\nData types in df2:")
print(df2[['substrate_cid', 'prod_cid', 'enzyme']].dtypes)

Data types in df:
Predecessor_CID     int64
Successor_CID       int64
Enzyme             object
dtype: object

Data types in df1:
InChI          object
PubChem_CID     int64
SMILES         object
dtype: object

Data types in df2:
substrate_cid    float64
prod_cid          object
enzyme            object
dtype: object


### Merging Datasets

In [17]:
df['Predecessor_Name'] = df['Predecessor_Name'].astype(str)
df['Successor_CID'] = df['Successor_CID'].astype(float)
df1['SubstanceName'] = df1['SubstanceName'].astype(str)
df1['PubChem_CID'] = df1['PubChem_CID'].astype(float)
df2['substrate_cid'] = df2['substrate_cid'].astype(str)
df2['prod_cid'] = df2['prod_cid'].astype(str)

# First merge
mergedDf1 = pd.merge(df, df1, 
                      left_on=['Successor_CID'], 
                      right_on=['PubChem_CID'], 
                      how='inner')

# Second merge
mergedDf = pd.merge(mergedDf1, df2, 
                     left_on=['Enzyme'], 
                     right_on=['enzyme'], 
                     how='inner')

# Initialization:

- Preprocessing dataset 
- Calculate Enzyme Frequencies
- Set the threshold for **high frequency enzymes**
- Group by **Transformation** [Type] and Enzyme, then count the occurrences
- Filter transformation types specific to **single enzymes**

In [18]:
%%time
enzymeCounter = Counter()
allPossiblePairs = set()
allPossibleEnzymes = set()
def updateSetsAndCounter(row):
    enzymes = row['Enzyme'].split('; ')
    enzymeCounter.update(enzymes)
    allPossiblePairs.add((row['Predecessor_CID'], row['Successor_CID']))
    allPossibleEnzymes.update(enzymes)
mergedDf.apply(updateSetsAndCounter, axis=1)
print("[INFO]: PHASE-1 DONE !")

enzymeFrequenciesDf = pd.DataFrame.from_dict(enzymeCounter, orient='index', columns=['Frequency']).reset_index()
enzymeFrequenciesDf.rename(columns={'index': 'Enzyme'}, inplace=True)
enzymeFrequenciesDf.sort_values(by='Frequency', ascending=False, inplace=True)
print("[INFO]: PHASE-2 DONE !")

highFrequencyEnzymes = set(enzymeFrequenciesDf[enzymeFrequenciesDf['Frequency'] >= 10]['Enzyme'])
print("[INFO]: PHASE-3 DONE !")

transformationEnzymeGroups = mergedDf.groupby(['Transformation', 'Enzyme']).size().reset_index(name='Counts')
enzymeSpecificTransformationsSet = set(
    transformationEnzymeGroups.groupby('Transformation')
    .filter(lambda x: len(x) == 1)['Transformation'])
print("[INFO]: PHASE-4 DONE !")

[INFO]: PHASE-1 DONE !
[INFO]: PHASE-2 DONE !
[INFO]: PHASE-3 DONE !
[INFO]: PHASE-4 DONE !
CPU times: total: 93.8 ms
Wall time: 98.2 ms


### Calculating weights based **high frequency enzymes** / **Transformations** / **Biosystem [Human]**

In [19]:
%%time
def calculateWeights(row):
    rowBasedWeight = 10 * (row['Enzyme'] in highFrequencyEnzymes)
    rowBasedWeight += 5 * (row['Transformation'] in enzymeSpecificTransformationsSet)
    rowBasedWeight += 3 * (row['Biosystem'] == 'Human')
    return rowBasedWeight
mergedDf['Row_Based_Weight'] = mergedDf.apply(calculateWeights, axis=1)
print("[INFO]: PHASE-5 DONE !")

def calculateChemicalStructureWeight(inchi, smiles):
    weight = 0
    molInchi = Chem.MolFromInchi(inchi)
    molSmiles = Chem.MolFromSmiles(smiles)
    if molInchi and molSmiles:
        mwInchi = Descriptors.MolWt(molInchi)
        mwSmiles = Descriptors.MolWt(molSmiles)
        fpInchi = Chem.RDKFingerprint(molInchi)
        fpSmiles = Chem.RDKFingerprint(molSmiles)
        tanimotoSimilarity = DataStructs.FingerprintSimilarity(fpInchi, fpSmiles)
        weight = mwInchi * tanimotoSimilarity + mwSmiles
    return weight
print("[INFO]: PHASE-6 DONE !")


Q = defaultdict(int)
N = 5 # at least have 5 pairs that catalyze with X enzyme

# Updating Q Model (Dictionary) [Q: QUBO Model]
def updateQ(row):
    try:
        enzymes = row['Enzyme'].split('; ')
        target = row['Successor_CID']
        predecessor = row['Predecessor_CID']
        weight1 = row['Row_Based_Weight']
        weight2 = calculateChemicalStructureWeight(row['InChI'], row['SMILES'])
        totalWeight = weight1 + weight2
        for enzyme in enzymes:
            enzyme = enzyme.strip()
            pair = f"{enzyme}_{predecessor}_{target}"
            Q[(pair, pair)] += -totalWeight
            for otherPredecessor, otherTarget in allPossiblePairs:
                otherPair = f"{enzyme}_{otherPredecessor}_{otherTarget}"
                Q[(pair, otherPair)] += 2 * N
    except Exception as e:
        print(f"An error occurred finding in chemical informatics: {e}")
        print(f"Skipping row: {row}")
mergedDf.apply(updateQ, axis=1)
print("[INFO]: PHASE-7 DONE !")

[INFO]: PHASE-5 DONE !
[INFO]: PHASE-6 DONE !


KeyboardInterrupt: 

## Initializing Constrains + Adjacencies for Q model

In [20]:
def updatePairConstraint(predecessor, target):
    pairsForThisReaction = [
        f"{enzyme.strip()}_{predecessor}_{target}"
        for enzyme in allPossibleEnzymes
    ]
    for i, pair1 in enumerate(pairsForThisReaction):
        for j, pair2 in enumerate(pairsForThisReaction):
            if i != j:
                Q[(pair1, pair2)] -= 2  # EXPERIMENTAL::Encourage at least one enzyme to catalyze each (predecessor, target)
print("[INFO]: PHASE-8 DONE !")

for predecessor, target in allPossiblePairs:
    updatePairConstraint(predecessor, target)
pairToIndex = {pair: i for i, pair in enumerate(allPossiblePairs)}
enzymeToIndex = {enzyme: j for j, enzyme in enumerate(allPossibleEnzymes)}
adjacencyMatrix = np.zeros((len(allPossiblePairs), len(allPossibleEnzymes)))
def updateAdjacencyMatrix(pair, i):
    for enzyme, j in enzymeToIndex.items():
        pairStr = f"{enzyme.strip()}_{pair[0]}_{pair[1]}"
        if (pairStr, pairStr) in Q:
            adjacencyMatrix[i, j] = 1            
print("[INFO]: PHASE-9 DONE !")

for pair, i in pairToIndex.items():
    updateAdjacencyMatrix(pair, i)
def refineConstraints(adjacencyMatrix):
    stronglyConnectedEnzymes = np.sum(adjacencyMatrix, axis=0) > 1
    stronglyConnectedPairs = np.sum(adjacencyMatrix, axis=1) > 1    
    for pair, i in pairToIndex.items():
        for enzyme, j in enzymeToIndex.items():
            pairStr = f"{enzyme.strip()}_{pair[0]}_{pair[1]}"
            qVal = Q.get((pairStr, pairStr), 0)
            if qVal:
                if stronglyConnectedEnzymes[j]:
                    Q[(pairStr, pairStr)] = qVal * 1.5  # Increase by 50%
                if stronglyConnectedPairs[i]:
                    Q[(pairStr, pairStr)] = qVal * 1.5  # Increase by 50%

refineConstraints(adjacencyMatrix)
print("[INFO]: PHASE-10 DONE !")

[INFO]: PHASE-8 DONE !
[INFO]: PHASE-9 DONE !
[INFO]: PHASE-10 DONE !


# Initialize QUBO [BQM] using Dimod

In [21]:
%%time
bqm = BinaryQuadraticModel.from_qubo(Q)

P:\Python\Lib\site-packages\dimod\binary\binary_quadratic_model.py:490: SyntaxWarning: invalid escape sequence '\o'
  """Binary-valued version of the binary quadratic model.
P:\Python\Lib\site-packages\dimod\binary\binary_quadratic_model.py:657: SyntaxWarning: invalid escape sequence '\s'
  """Add a linear constraint as a quadratic objective.
P:\Python\Lib\site-packages\dimod\binary\binary_quadratic_model.py:714: SyntaxWarning: invalid escape sequence '\s'
  """Add a linear inequality constraint as a quadratic objective.
P:\Python\Lib\site-packages\dimod\binary\binary_quadratic_model.py:2615: SyntaxWarning: invalid escape sequence '\{'
  """Return a binary quadratic model with a single binary variable.
P:\Python\Lib\site-packages\dimod\binary\binary_quadratic_model.py:2641: SyntaxWarning: invalid escape sequence '\o'
  """Yield binary quadratic models, each with a single binary variable.
P:\Python\Lib\site-packages\dimod\binary\binary_quadratic_model.py:2677: SyntaxWarning: invalid esc

KeyboardInterrupt: 

## Applying Constrains/Adjancencies to BQM

In [22]:
for i, (predecessor, target) in enumerate(allPossiblePairs):
    for j, enzyme in enumerate(allPossibleEnzymes):
        pairStr = f"{enzyme.strip()}_{predecessor}_{target}"
        if adjacencyMatrix[i, j] == 1:
            if (pairStr, pairStr) in Q:
                bqm.set_linear(pairStr, Q[(pairStr, pairStr)])

## Compute using DIMOD [Simulated Annealing Sampler]

In [23]:
%%time
sampler = SimulatedAnnealingSampler()
response = sampler.sample(bqm, num_reads=1)  # My CPU Couldn't handle more than 1 num_reads :\
bestSol = response.first.sample
bestEng = response.first.energy
#print("Best Solution:", bestSol)
#print("Energy:", bestEng)

KeyboardInterrupt: 

## - Compute using DYNEX

In [24]:
'''
%%time
model = dynex.BQM(bqm);
sampler = dynex.DynexSampler(model,  mainnet=True, description='AndreasRossos-Enzyme-TargetPrediction')
sampleset = sampler.sample(num_reads=10000, annealing_time = 1000, debugging=True);
bestSol = sampler.dimod_assignments.first.sample;
'''

"\n%%time\nmodel = dynex.BQM(bqm);\nsampler = dynex.DynexSampler(model,  mainnet=True, description='AndreasRossos-Enzyme-TargetPrediction')\nsampleset = sampler.sample(num_reads=10000, annealing_time = 1000, debugging=True);\nbestSol = sampler.dimod_assignments.first.sample;\n"

In [25]:
#print('DYNEX RESULT:',bestSol)

# Loading Best Solution

In [26]:
optimalPairs = [pair for pair, value in bestSol.items() if value == 1]
optimalTransformations = {}

for pair in optimalPairs:
    _, predecessor, target = pair.split('_')
    predQuery = df1.loc[df1['PubChem_CID'] == int(predecessor), 'SMILES']
    targetQuery = df1.loc[df1['PubChem_CID'] == float(target), 'SMILES']
    predecessorSmiles = predQuery.iloc[0] if not predQuery.empty else 'Not Found'
    targetSmiles = targetQuery.iloc[0] if not targetQuery.empty else 'Not Found'
    optimalTransformations[predecessorSmiles] = targetSmiles

    #print(f"Optimal transformation: {predecessorSmiles} -> {targetSmiles}")

NameError: name 'bestSol' is not defined

## - Visualization

In [ ]:
def Visualize(predecessorSmiles):
    targetSmiles = optimalTransformations.get(predecessorSmiles, "Not found")
    if targetSmiles == "Not found":
        print("Predecessor SMILES not found in the optimal transformations.")
        return
    # Drawing predecessor molecule
    predMol = Chem.MolFromSmiles(predecessorSmiles)
    predImg = Draw.MolToMPL(predMol, size=(300, 300), kekulize=True)
    plt.title("Predecessor Molecule")
    plt.axis("off")
    plt.show()
    
    # Drawing target molecule
    targetMol = Chem.MolFromSmiles(targetSmiles)
    targetImg = Draw.MolToMPL(targetMol, size=(300, 300), kekulize=True)
    plt.title("Predicted Target Molecule")
    plt.axis("off")
    plt.show()

predecessorSmiles = "C[C@]12CCC(=O)C=C1CC[C@H]1[C@@H]3CC[C@@H]([C@@]3(C)CC[C@H]21)O" # EXAMPLE 
Visualize(predecessorSmiles)